In [1]:
from langgraph_sdk import get_client

# Connect via SDK
url_for_cli_deployment = "http://localhost:8123"
client = get_client(url=url_for_cli_deployment)

In [2]:
from langgraph.pregel.remote import RemoteGraph
from langchain_core.messages import convert_to_messages
from langchain_core.messages import HumanMessage, SystemMessage

# Connect via remote graph
url = "http://localhost:8123"
graph_name = "task_maistro" 
remote_graph = RemoteGraph(graph_name, url=url)

In [5]:
# Create a thread
thread = await client.threads.create()
thread

{'thread_id': '643c952d-8b54-4b96-9b66-a619cd8bda7c',
 'created_at': '2025-02-03T20:09:43.760754+00:00',
 'updated_at': '2025-02-03T20:09:43.760754+00:00',
 'metadata': {},
 'status': 'idle',
 'config': {},
 'values': None,
 'interrupts': {}}

In [6]:
# Ensure we've created some ToDos and saved them to my user_id
user_input = "Add a ToDo to finish booking travel to Hong Kong by end of next week. Also, add a ToDo to call parents back about Thanksgiving plans."
config = {"configurable": {"user_id": "Test"}}
graph_name = "task_maistro" 
run = await client.runs.create(thread["thread_id"], graph_name, input={"messages": [HumanMessage(content=user_input)]}, config=config)

In [7]:
# Kick off a new thread and a new run
thread = await client.threads.create()
user_input = "Give me a summary of all ToDos."
config = {"configurable": {"user_id": "Test"}}
graph_name = "task_maistro" 
run = await client.runs.create(thread["thread_id"], graph_name, input={"messages": [HumanMessage(content=user_input)]}, config=config)

In [8]:

# Check the run status
print(await client.runs.get(thread["thread_id"], run["run_id"]))

{'run_id': '1efe26ae-4036-6a59-b77e-003811ea8d47', 'thread_id': '42900ecc-d37c-4d45-8a53-9c35f3d5de19', 'assistant_id': 'ea4ebafa-a81d-5063-a5fa-67c755d98a21', 'created_at': '2025-02-03T20:10:19.163044+00:00', 'updated_at': '2025-02-03T20:10:19.163044+00:00', 'metadata': {'assistant_id': 'ea4ebafa-a81d-5063-a5fa-67c755d98a21'}, 'status': 'success', 'kwargs': {'input': {'messages': [{'id': None, 'name': None, 'type': 'human', 'content': 'Give me a summary of all ToDos.', 'example': False, 'additional_kwargs': {}, 'response_metadata': {}}]}, 'config': {'metadata': {'created_by': 'system', 'assistant_id': 'ea4ebafa-a81d-5063-a5fa-67c755d98a21'}, 'configurable': {'run_id': '1efe26ae-4036-6a59-b77e-003811ea8d47', 'user_id': 'Test', 'graph_id': 'task_maistro', 'thread_id': '42900ecc-d37c-4d45-8a53-9c35f3d5de19', 'assistant_id': 'ea4ebafa-a81d-5063-a5fa-67c755d98a21', 'langgraph_auth_user': None, 'langgraph_auth_user_id': '', 'langgraph_auth_permissions': []}}, 'command': None, 'webhook': Non

In [9]:
# Wait until the run completes
await client.runs.join(thread["thread_id"], run["run_id"])
print(await client.runs.get(thread["thread_id"], run["run_id"]))

{'run_id': '1efe26ae-4036-6a59-b77e-003811ea8d47', 'thread_id': '42900ecc-d37c-4d45-8a53-9c35f3d5de19', 'assistant_id': 'ea4ebafa-a81d-5063-a5fa-67c755d98a21', 'created_at': '2025-02-03T20:10:19.163044+00:00', 'updated_at': '2025-02-03T20:10:19.163044+00:00', 'metadata': {'assistant_id': 'ea4ebafa-a81d-5063-a5fa-67c755d98a21'}, 'status': 'success', 'kwargs': {'input': {'messages': [{'id': None, 'name': None, 'type': 'human', 'content': 'Give me a summary of all ToDos.', 'example': False, 'additional_kwargs': {}, 'response_metadata': {}}]}, 'config': {'metadata': {'created_by': 'system', 'assistant_id': 'ea4ebafa-a81d-5063-a5fa-67c755d98a21'}, 'configurable': {'run_id': '1efe26ae-4036-6a59-b77e-003811ea8d47', 'user_id': 'Test', 'graph_id': 'task_maistro', 'thread_id': '42900ecc-d37c-4d45-8a53-9c35f3d5de19', 'assistant_id': 'ea4ebafa-a81d-5063-a5fa-67c755d98a21', 'langgraph_auth_user': None, 'langgraph_auth_user_id': '', 'langgraph_auth_permissions': []}}, 'command': None, 'webhook': Non

In [10]:
user_input = "What ToDo should I focus on first."
async for chunk in client.runs.stream(thread["thread_id"], 
                                      graph_name, 
                                      input={"messages": [HumanMessage(content=user_input)]},
                                      config=config,
                                      stream_mode="messages-tuple"):

    if chunk.event == "messages":
        print("".join(data_item['content'] for data_item in chunk.data if 'content' in data_item), end="", flush=True)

You should focus on "Call parents back about Thanksgiving plans" first. It doesn't have a specific deadline, but it's a shorter task with an estimated time to complete of just 15 minutes. Plus, it's always good to finalize plans with family early!

In [11]:
thread_state = await client.threads.get_state(thread['thread_id'])
for m in convert_to_messages(thread_state['values']['messages']):
    m.pretty_print()

================================ Human Message =================================

Give me a summary of all ToDos.
================================== Ai Message ==================================

Here's a summary of your current ToDo list:

1. **Task**: Finish booking travel to Hong Kong
   - **Status**: Not started
   - **Deadline**: February 14, 2025
   - **Solutions**:
     - Check flight prices on Skyscanner
     - Book hotel through Booking.com
     - Arrange airport transfer
   - **Estimated Time to Complete**: 120 minutes

If you need any updates or changes, feel free to let me know!
================================ Human Message =================================

What ToDo should I focus on first.
================================== Ai Message ==================================

You should focus on "Call parents back about Thanksgiving plans" first. It doesn't have a specific deadline, but it's a shorter task with an estimated time to complete of just 15 minutes. Plus, it's alwa